### Pablo Chantada Saborido (pablo.chantada@udc.es)
### José Romero Conde (j.rconde@udc.es)

# CARGA DE DATOS

In [ ]:
from Utils import DatasetProcess, reconstruction_plot, anomaly_report, plot_atipicos, plot_model
from ConvModel import ConvModel
from AutoEncoder import TwoStepAutoEncoder, TwoStepClassifier, TwoStepTraining, OneStepAutoencoder, OneStepTraining
from OneClass import AnomalyDetector
from Contrastive import ContrastiveModel, SemiSupervisedContrastiveModel

In [ ]:
(x_train_full, y_train_full), (x_test, y_test) = DatasetProcess.load()
(unlabeled_train, x_train, y_train), (x_val, y_val), (x_test, y_test) = DatasetProcess.hold_out(
    train=(x_train_full, y_train_full),
    test=(x_test, y_test),
    validation_size=2000
)

In [ ]:
# Verificar las dimensiones
print(f"Datos no etiquetados: {unlabeled_train.shape}")
print(f"Datos etiquetados entrenamiento: {x_train.shape}")
print(f"Etiquetas entrenamiento: {y_train.shape}")
print(f"Datos validación: {x_val.shape}")
print(f"Etiquetas validación: {y_val.shape}")
print(f"Datos prueba: {x_test.shape}")
print(f"Etiquetas prueba: {y_test.shape}")

# EJERCICIO 1

Entrena un modelo, creado sobre TensorFlow, haciendo uso únicamente de las instancias etiquetadas de entrenamiento. Dicho modelo debe de tener al menos cuatro capas densas y/o convolucionales.

Responde a las siguientes preguntas:

1. ¿Qué red has escogido? ¿Por qué? ¿Cómo la has entrenado?
2. ¿Cuál es el rendimiento del modelo en entrenamiento? ¿Y en prueba?
3. ¿Qué conclusiones sacas de los resultados detallados en el punto anterior?

In [ ]:
model_base = ConvModel()
history_base = model_base.fit(
    x_train, 
    y_train,
    validation_data=(x_val, y_val),
    batch_size=128,
    epochs=100
)

In [ ]:
# Evalúa con el conjunto de prueba
base_accuracy = model_base.score(x_test, y_test)
print(f"Accuracy en conjunto de prueba: {base_accuracy}")

model_base.plot(history_base)

# EJERCICIO 2

Entrena el mismo modelo, incorporando las instancias no etiquetadas de entrenamiento mediante la técnica de auto-aprendizaje. Opcionalmente, se ponderará cada instancia de entrada en función de su calidad (o certeza).

Responde a las siguientes preguntas:
1. ¿Qué parámetros has definido para el entrenamiento?
2. ¿Cuál es el rendimiento del modelo en entrenamiento? ¿Y en prueba?
3. ¿Se mejoran los resultados obtenidos en el Ejercicio 1?
4. ¿Qué conclusiones sacas de los resultados detallados en los puntos anteriores?

In [ ]:
# Función para crear modelos consistentes durante self-training
def create_model():
    return ConvModel(
        learning_rate=0.0005,  # Learning rate reducido para fine-tuning
        dropout_prob=0.25,     
        l2_lambda=0.005        
    )

# Aplica self-training con datos no etiquetados
final_model = ConvModel.self_training_v2(
    model_func=create_model,
    x_train=x_train,
    y_train=y_train,  
    unlabeled_data=unlabeled_train,
    validation_data=(x_val, y_val),
    thresh=0.85,             
    train_epochs=4,
    verbose=1
)

# Evalúa el modelo final
final_accuracy = final_model.score(x_test, y_test)
print(f"Accuracy del modelo final con self-training: {final_accuracy}")
print(f"Mejora respecto al modelo base: {final_accuracy - base_accuracy:.4f}")

---
Como se han cambiado los datos los volvemos a cargarlos para una comparación justa

In [ ]:
(x_train_full, y_train_full), (x_test, y_test) = DatasetProcess.load()
(unlabeled_train, x_train, y_train), (x_val, y_val), (x_test, y_test) = DatasetProcess.hold_out(
    train=(x_train_full, y_train_full),
    test=(x_test, y_test),
    validation_size=2000
)	

In [ ]:
# Aplica self-training con datos no etiquetados
final_model = ConvModel.self_training_v2(
    model_func=create_model,
    x_train=x_train,
    y_train=y_train,  
    unlabeled_data=unlabeled_train,
    validation_data=(x_val, y_val),
    thresh=0.6,             
    train_epochs=4,
    verbose=1
)

# Evalúa el modelo final
final_accuracy = final_model.score(x_test, y_test)
print(f"Accuracy del modelo final con self-training: {final_accuracy}")
print(f"Mejora respecto al modelo base: {final_accuracy - base_accuracy:.4f}")

In [ ]:
unlabeled_train, x_train, y_train, x_val, y_val, x_test, y_test, one_hot_train, one_hot_val, one_hot_test = DatasetProcess.load_dataset()

# EJERCICIO 3

Entrena un modelo de aprendizaje semisupervisado de tipo autoencoder en dos pasos (primero el autoencoder, después el clasificador). La arquitectura del encoder debe ser exactamente la misma que la definida en los Ejercicios 1 y 2, a excepción del último bloque de capas.

Responde a las siguientes preguntas:
1. ¿Cuál es la arquitectura del modelo? ¿Y sus hiperparámetros?
2. ¿Cuál es el rendimiento del modelo en entrenamiento? ¿Y en prueba?
3. ¿Se mejoran los resultados obtenidos en los Ejercicios 1 y 2?
4. ¿Qué conclusiones sacas de los resultados detallados en los puntos anteriores?

In [ ]:
autoencoder = TwoStepAutoEncoder(
                                input_shape=unlabeled_train[0].shape,
                                learning_rate=0.01,
                                l2_lambda=0.0005,
                                dropout_prob=0.1)
classifier = TwoStepClassifier(
                              l2_lambda=0.0005,
                              dropout_prob=0.05,
                               learning_rate=0.05)


In [ ]:
TwoStepTraining(autoencoder=autoencoder, 
                classifier=classifier, 
                x_train=x_train, 
                y_train=one_hot_train, 
                unlabeled_train=unlabeled_train, 
                validation_data=(x_val, one_hot_val),
                batch_size_autoencoder=512,
                epochs_autoencoder=15,
                batch_size_classifier=1024,
                epochs_classifier=50)

In [ ]:
reconstruction_plot(autoencoder, x_test)

In [ ]:
code = autoencoder.get_encoded_data(x_test)
test_accuracy = classifier.score(code, y_test)
print(f"Accuracy en conjunto de prueba: {test_accuracy}")
print(f"Mejora respecto al modelo base: {test_accuracy - base_accuracy:.4f}")

# EJERCICIO 4

Entrena un modelo de aprendizaje semisupervisado de tipo autoencoder en un paso (autoencoder y clasificador al mismo tiempo). La arquitectura del autoencoder será la misma que la definida en el Ejercicio 3, y la combinación de encoder y clasificador será igual a la arquitectura definida en el
Ejercicio 1.

Responde a las siguientes preguntas:
1. ¿Cuál es la arquitectura del modelo? ¿Y sus hiperparámetros?
2. ¿Cuál es el rendimiento del modelo en entrenamiento? ¿Y en prueba?
3. ¿Se mejoran los resultados obtenidos en los ejercicios anteriores?
4. ¿Qué conclusiones sacas de los resultados detallados en los puntos anteriores?

In [ ]:
one_step_autoencoder = OneStepAutoencoder(input_shape=unlabeled_train[0].shape,
                                learning_rate=0.0035,
                                decoder_extra_loss_weight = 0.5,
                                l2_lambda=0.00005,
                                dropout_prob=0.05)

In [ ]:
h = OneStepTraining(one_step_autoencoder, 
                    x_train=x_train, 
                    y_train=one_hot_train, 
                    unlabeled_train=unlabeled_train,
                    batch_size=512,
                    epochs=30,
                    patience=4)

In [ ]:
reconstruction_plot(one_step_autoencoder, x_test)

In [ ]:
test_accuracy = one_step_autoencoder.score(x_test, y_test)
print(f"Accuracy en conjunto de prueba: {test_accuracy}")
print(f"Mejora respecto al modelo base: {test_accuracy - base_accuracy:.4f}")

# EJERCICIO 5

Repite el mismo entrenamiento de los Ejercicios 1-4, pero eliminando las instancias no etiquetadas
más atípicas con respecto a los datos etiquetados. Se cumplirán los siguientes puntos:
- La arquitectura de la red de clasificación en una clase será la misma a la utilizada en el
clasificador del Ejercicio 1, a excepción de la capa de salida.
- Utiliza la técnica explicada en el Notebook 5, usando un valor de 𝑣 = 0,9.

Responde a la siguiente pregunta:
1. ¿Se mejoran los resultados con respecto a los anteriores ejercicios? ¿Qué conclusiones sacas de estos resultados?

In [ ]:
anomalyDetector = AnomalyDetector(input_shape=(32,32,3), 
                        nu=.9,
                        l2_lambda=0.0,
                        learning_rate=0.0001,
                        dropout_prob=0.0)
anomalyDetector.fit(x_train, 
          batch_size=256, 
          epochs=35, 
          delta=.025, 
          steps=100)

In [ ]:
filtered_unlabeled_train, is_typical = anomaly_report(anomalyDetector, unlabeled_train)

In [ ]:
plot_atipicos(is_typical, unlabeled_train)

---
Ahora que tenemos los datos filtrados, repetimos

---

In [ ]:
(x_train_full, y_train_full), (x_test, y_test) = DatasetProcess.load()
(unlabeled_train, x_train, y_train), (x_val, y_val), (x_test, y_test) = DatasetProcess.hold_out(
    train=(x_train_full, y_train_full),
    test=(x_test, y_test),
    validation_size=2000
)

### Ejercicio 2 con datos filtrados
model_self_filtered = ConvModel.self_training_v2(
    model_func=create_model,
    x_train=x_train,
    y_train=y_train,
    unlabeled_data=filtered_unlabeled_train,
    validation_data=(x_val, y_val),
    thresh=0.85,
    train_epochs=4
)

# Evaluar los modelos filtrados
test_accuracy_self_filtered = model_self_filtered.score(x_test, y_test)
print(f"Accuracy en conjunto de prueba (self-training filtrado): {test_accuracy_self_filtered}")
print(f"Mejora respecto al modelo base: {test_accuracy_self_filtered - base_accuracy:.4f}")

In [ ]:
unlabeled_train, x_train, y_train, x_val, y_val, x_test, y_test, one_hot_train, one_hot_val, one_hot_test = DatasetProcess.load_dataset()

In [ ]:
# Ejercicio 3 con datos filtrados
autoencoder_filtered = TwoStepAutoEncoder(
    input_shape=filtered_unlabeled_train[0].shape,
    learning_rate=0.006,
    l2_lambda=0.0005,
    dropout_prob=0.1
)

classifier_filtered = TwoStepClassifier(
    l2_lambda=0.0005,
    dropout_prob=0.05,
    learning_rate=0.05
)

history_two_step_filtered = TwoStepTraining(
    autoencoder=autoencoder_filtered, 
    classifier=classifier_filtered, 
    x_train=x_train, 
    y_train=one_hot_train, 
    unlabeled_train=filtered_unlabeled_train, 
    validation_data=(x_val, one_hot_val),
    batch_size_autoencoder=256,
    epochs_autoencoder=15,
    batch_size_classifier=256,
    epochs_classifier=50
)

# Evaluar modelos

code = autoencoder_filtered.get_encoded_data(x_test)
test_accuracy = classifier_filtered.score(code, y_test)
print(f"Accuracy del modelo one-step filtrado: {test_accuracy}")
print(f"Mejora respecto al modelo base: {test_accuracy - base_accuracy:.4f}")

reconstruction_plot(autoencoder_filtered, x_test)

In [ ]:
# Ejercicio 4 con datos filtrados
one_step_autoencoder_filtered = OneStepAutoencoder(
    input_shape=filtered_unlabeled_train[0].shape,
    learning_rate=0.0015,
    decoder_extra_loss_weight=0.45,
    l2_lambda=0.00005,
    dropout_prob=0.05
)

history_one_step_filtered = OneStepTraining(
    one_step_autoencoder_filtered, 
    x_train=x_train, 
    y_train=one_hot_train, 
    unlabeled_train=filtered_unlabeled_train,
    batch_size=256,
    epochs=20,
    patience=10
)
# Evaluar modelos
reconstruction_plot(one_step_autoencoder_filtered, x_test)

test_accuracy_one_step_filtered = one_step_autoencoder_filtered.score(x_test, y_test)
print(f"Accuracy del modelo one-step filtrado: {test_accuracy_one_step_filtered}")
print(f"Mejora respecto al modelo base: {test_accuracy_one_step_filtered - base_accuracy:.4f}")

# EJERCICIO 6

Repite los Ejercicios 3-5 cambiando el autencoder por la técnica definida en el apartado “Hay vida más allá del autoencoder” del Notebook 4. Contesta a las preguntas de dichos ejercicios. Se cumplirán los siguientes puntos:

1. La arquitectura de la red será igual a la parte encoder del autencoder definido en los
ejercicios anteriores.
2. El modelo debe entrenar correctamente.

In [ ]:
# ejercicio 3 two step con todos los datos

cModel = ContrastiveModel(unlabeled_train[0].shape, 
                          learning_rate=0.05, 
                          lambda_param=.9,
                          l2_lambda=0.0001,
                          dropout_prob=0.005)
classifier = TwoStepClassifier(
                              l2_lambda=0.0005,
                              dropout_prob=0.01,
                               learning_rate=0.05)


In [ ]:
unlabeled_train, x_train, y_train, x_val, y_val, x_test, y_test, one_hot_train, one_hot_val, one_hot_test = DatasetProcess.load_dataset()

In [ ]:
x_coded = autoencoder.get_encoded_data_batched(x_train)
x_val_coded = autoencoder.get_encoded_data_batched(x_val)

In [ ]:
TwoStepTraining(autoencoder=cModel, 
                classifier=classifier, 
                x_train=x_train, 
                y_train=one_hot_train, 
                unlabeled_train=unlabeled_train, 
                validation_data=(x_val, one_hot_val),
                batch_size_autoencoder=128,
                epochs_autoencoder=15,
                batch_size_classifier=128,
                epochs_classifier=50, 
                contrastive=True)

# Evaluar modelos

code = cModel.get_encoded_data(x_test)
test_accuracy = classifier.score(code, y_test)
print(f"Accuracy del modelo one-step filtrado: {test_accuracy}")
print(f"Mejora respecto al modelo base: {test_accuracy - base_accuracy:.4f}")

reconstruction_plot(autoencoder_filtered, x_test)

In [ ]:
# ejercicio 4 ONE STEP con todos los datos

cSSLModel = SemiSupervisedContrastiveModel(unlabeled_train[0].shape, 
                          learning_rate=0.15, 
                          lambda_param=.9,
                          l2_lambda=0.0001,
                          dropout_prob=0.001,
                          lambda_supervised=1.0)


cSSLModel.train(
    X_unlabeled=unlabeled_train,  
    X_labeled=x_train,        
    y_labeled=y_train,       
    epochs=20,
    batch_size= 256
)

cSSLModel.plot_training_history()
test_accuracy = cSSLModel.score(x_test, y_test)
print(f"Accuracy del modelo one-step filtrado contrastivo con todos los datos: {test_accuracy}")
print(f"Mejora respecto al modelo base: {test_accuracy - base_accuracy:.4f}")
reconstruction_plot(cSSLModel, x_test)
cSSLModel.confusion_matrix(x_test, y_test)

In [ ]:
n_samples = 100
test_samples = x_test[:n_samples].reshape(n_samples,32,32,3)

cSSLModel.plot_similarity_matrix(test_samples, n_samples=n_samples)

---

In [ ]:
# ejercicio 3 two step con los datos filtrados

cModel = ContrastiveModel(unlabeled_train[0].shape, 
                          learning_rate=0.05, 
                          lambda_param=.9,
                          l2_lambda=0.001,
                          dropout_prob=0.001)
classifier = TwoStepClassifier(
                              l2_lambda=0.0005,
                              dropout_prob=0.05,
                               learning_rate=0.05)

In [ ]:
TwoStepTraining(autoencoder=cModel, 
                classifier=classifier, 
                x_train=x_train, 
                y_train=one_hot_train, 
                unlabeled_train=filtered_unlabeled_train, # <-
                validation_data=(x_val, one_hot_val),
                batch_size_autoencoder=256,
                epochs_autoencoder=15,
                batch_size_classifier=256,
                epochs_classifier=50, 
                contrastive=True)

In [ ]:
# Evaluacion ejercicio 3 two step con los datos filtrados

code = cModel.get_encoded_data(x_test)
test_accuracy = classifier.score(code, y_test)
print(f"Accuracy del modelo two-step filtrado: {test_accuracy}")
print(f"Mejora respecto al modelo base: {test_accuracy - base_accuracy:.4f}")

reconstruction_plot(cModel, x_test)

In [ ]:
# ejercicio 4 ONE STEP con los datos filtrados

cSSLModel = SemiSupervisedContrastiveModel(unlabeled_train[0].shape, 
                          learning_rate=0.05, 
                          lambda_param=.9,
                          l2_lambda=0.001,
                          dropout_prob=0.001,
                          lambda_supervised=1.0)


cSSLModel.train(
    X_unlabeled=filtered_unlabeled_train,  
    X_labeled=x_train,        
    y_labeled=y_train,       
    epochs=20,
    batch_size=256
)



cSSLModel.plot_training_history()


test_accuracy = cSSLModel.score(x_test, y_test)
print(f"Accuracy del modelo one-step filtrado contrastivo con datos filtrados: {test_accuracy}")
print(f"Mejora respecto al modelo base: {test_accuracy - base_accuracy:.4f}")

In [ ]:
n_samples = 100
test_samples = x_test[:n_samples].reshape(n_samples,32,32,3)

cSSLModel.plot_similarity_matrix(test_samples, n_samples=n_samples)